In [1]:
#import library
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import edward2 as ed
tfd = tfp.distributions
import scipy.stats as st
from mpl_toolkits.mplot3d import Axes3D
import math

import seaborn as sns
import pandas as pd
%matplotlib inline

\begin{equation}
y | \boldsymbol{\theta} \sim \mathcal{N}\left(\mu_{y}, \sigma_{y}^{2}\right), \quad \mu_{y}:=\sum_{k=1}^{\lceil D / 2\rceil} \theta_{2 k-1}+\sum_{k=1}^{\lfloor D / 2\rfloor} \theta_{2 k}^{2}
\end{equation}

\begin{equation}
\theta_{i} \stackrel{\text { iid }}{\sim} \mathcal{N}\left(0, \sigma_{\theta}^{2}\right)
\end{equation}

In [26]:
# define dimentions D

D = np.int(4)

mu= np.zeros(D)

cov = np.eye(D)

\begin{equation}
\theta_{1}+\theta_{2}^{2}+\theta_{3}+\theta_{4}^{2}
\end{equation}


In [52]:
#define forward function
def f(D,theta):# theta should be vector in D dimentions
    Sum = 0
    for i in range(1,D//2+1):
        Sum += theta[2*i-2]+theta[2*i-1]**2
    return Sum
theta = [0,0,0,0]

f(4,theta)

0

In [32]:
## initial the model

# generate data
c = 0 # mean of observations
N = 100 # number of observation data
sigma2y = 1 # standard deviation of observation data
## generate the observation data
y_ = np.random.normal(loc =c, scale = sigma2y, size = N)
Data = tf.convert_to_tensor(y_,dtype = tf.float32)


In [67]:
mu = tf.convert_to_tensor(mu,dtype = tf.float32)
cov = tf.convert_to_tensor(cov,dtype=tf.float32)

<tf.Tensor: id=978, shape=(), dtype=float32, numpy=0.025330298>

In [71]:
## define the log probability
def joint_log_prob(Data,theta,Dimention):
    # define random variables prior
    mvn = tfd.MultivariateNormalFullCovariance(
            loc = mu,
            covariance_matrix= cov)
    # define likelihood
    y = tfd.Normal(loc = f(Dimention,theta), scale = sigma2y)
    # return the posterior probability
    return (tf.exp(mvn.prob(theta)
          *tf.reduce_sum(y.prob(Data))))

In [72]:
joint_log_prob(Data,theta,D)

<tf.Tensor: id=1222, shape=(), dtype=float32, numpy=2.0708382>

In [ ]:
x_ = np.mgrid[-2:2:.1, -2:2:.1]
pos = np.empty(x.shape + (2,),dtype = np.float32) 
pos[:, :, 0] = x; pos[:, :, 1] = y
likelihood = np.empty(x.shape)
for i in range(400):
    for j in range(400):
        likelihood[i][j] = likelihood_prob(D,pos[i][j][0],pos[i][j][1])
        
